In [ ]:
import os
import glob
import json
from pathlib import Path
import tkinter
from tkinter import filedialog
from pathlib import Path
import cv2
import numpy as np
import random

In [ ]:
image_width = 0
image_height = 0
animal = ""

label_codes = {}

label_codes["A1_구진_플라크"] = 0
label_codes["A2_비듬_각질_상피성잔고리"] = 1
label_codes["A3_태선화_과다색소침착"] = 2
label_codes["A4_농포_여드름"] = 3
label_codes["A5_미란_궤양"] = 4
label_codes["A6_결절_종괴"] = 5
save= True
debug = False
visualize = False

classes = len(label_codes)

issue_files_list = []

colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(classes)]

In [ ]:
def plot_one_box(x, image, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (image.shape[0] + image.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(image, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(image, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(image, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    
    cv2.imshow("labeled",image)
    cv2.waitKey(30)
    cv2.destroyAllWindows()

In [ ]:
def draw_boxes(label,x,y,w,h, image):
    
        global image_width, image_height
        
        #print("image_width",image_width)
        #print("image_height",image_height)
    
        x_center, y_center, w, h = float(x)*image_width, float(y)*image_height, float(w)*image_width, float(h)*image_height
        x1 = round(x_center-w/2)
        y1 = round(y_center-h/2)
        x2 = round(x_center+w/2)
        y2 = round(y_center+h/2)  
        
        class_idx = label_codes[label]
        
        plot_one_box([x1,y1,x2,y2], image, color=colors[class_idx], label=label, line_thickness=2)        
        

In [ ]:
def process_file(m_path,inp,out):
    
    global image_width, image_height, animal
    
    im_ext = ".jpg"
    
    json_labels,json_meta = parse_file(inp)    
    
    size = len(json_labels)
    count = 0   
    
    if debug:
        for lab in json_labels:
            print(lab)
    
    if size > 2:
        print("MORE THAN ONE LABEL")
        print("json_labels",json_labels)
        #input()
        
    bboxes = []
    
    for json_label in json_labels:
        if "box" in json_label: 
            bboxes.append(json_label)
            
    boxes_size= len(bboxes)    
        
    for bbox in bboxes:  
        
        count += 1

        if debug:
            print("json label",json_label)

        box = bbox["box"]

        label = box["label"]
        label = "".join(label.split())
        #print("label",label)
        bounding_b = box["location"]
        bbox_size = len(bounding_b)

        if bbox_size > 1:
            print("MORE THAN ONE BBOX FOUND")
            print("bboxes",bounding_b)
            #input()

        bounding_b = bounding_b[0]

        #img_name = json_meta["Raw data ID"]
        
        img_file = str(Path(os.path.splitext(inp)[0]+im_ext))
        
        #img_file = str(Path(m_path+"/"+img_name))
        if debug:
            print(img_file)
        #print(inp)

        #frame = cv2.imread(img_file)
        frame = cv2.imdecode(np.fromfile(img_file, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
        
        image_height, image_width, _ = frame.shape
        
        #print("frame",image_height, image_width)

        xmin = bounding_b["x"]
        ymin = bounding_b["y"]
        width = bounding_b['width']
        height = bounding_b['height']
        
        
        if width < 10 or height < 10:
            issue_files_list.append(inp)

        x = (xmin+(width/2))/image_width
        y = (ymin+(height/2))/image_height 
        W = width/image_width
        H = height/image_height
        
        if visualize:
            draw_boxes(label,x,y,W,H, frame)

        if animal == "반려묘":
            yolo_label = 0
        elif animal == "반려견":
            yolo_label = label_codes[label]
        else:
            print("ERROR ! no other animal should be here")
            input()

        ## saving section ##

        if save:
            if not(os.path.exists(out)):
                with open(out, 'a') as yolofile:
                    if count == boxes_size:
                        yolofile.write(f'{yolo_label} {x} {y} {W} {H}')
                    else:
                         yolofile.write(f'{yolo_label} {x} {y} {W} {H}\n')
                #print(f'{yolo_label} {x} {y} {W} {H}')
                print("Saved file:",out)
            else:
                print("file exists:",out)

            
       


In [ ]:
def parse_file(inp):
    json_obj = json.load(open(inp,encoding="utf-8"))
    json_labels = json_obj["labelingInfo"]
    if debug:
        print("json labels",json_labels)
    json_meta = json_obj["metaData"]
    return json_labels,json_meta

In [ ]:
def process_folder(path_files):
    ext = ".json"
    
    files = os.listdir(path_files)   
          
    for file in files:
        if file.endswith(ext):
            filename = os.path.splitext(file)[0]            
            input_file = str(Path(path_files +"/" + file))            
            output_file =  str(Path(path_files +"/" + filename+".txt"))
            
            process_file(path_files,input_file,output_file)
    

In [ ]:
def process_folders(animal,path_folds):   
    
    if animal == "반려견":
        folders = os.listdir(path_folds)    
        #folders = ["A3_태선화_과다색소침착"]
    elif animal == "반려묘":
        folders = ["A2_비듬_각질_상피성잔고리"]
    
    for folder in folders:
        folder_path = str(Path(path_folds+"/"+folder))
        print("folder_path",folder_path)
        process_folder(folder_path)


In [ ]:
def process_all(main_path):
    
    global animal
    
    folders_1 = ["1.Training", "2.Validation"]
    folders_2 = ["반려견","반려묘"]
    
    for folder_1 in folders_1:
        #folder_3 = "TL01" if folder_1 == "1.Training" else "VL01"
        for folder_2 in folders_2:  
            animal = folder_2
            #current_path = str(Path(main_path + "/"+folder_1+"/2_라벨링데이터/"+folder_3+"/"+ folder_2+ "/피부/일반카메라/유증상/"))
            current_path = str(Path(main_path + "/"+folder_1+"/2_라벨링데이터/"+ folder_2+ "/피부/일반카메라/유증상/"))
            print(current_path)
            process_folders(animal,current_path)

            

In [ ]:
def run_code():
    root = tkinter.Tk()
    dir_path = filedialog.askdirectory(parent=root,initialdir="/",title='Please select the "01.데이터" directory')
    print("\ndir_path : ", dir_path)
    root.destroy()
    #dir_path = "C:\\Users\\db-lab\\Downloads\\152.반려동물 피부질환 데이터\\01.데이터"
    if dir_path == "":
        print("no folder was selected")
    elif dir_path.endswith("01.데이터"):
                
        process_all(dir_path)
        print()
        for issue in issue_files_list:
            print(issue)
                
    else:
        print("Please make sure you select the '01.데이터' directory in the dataset")
        
        

In [ ]:
if __name__ == "__main__":
    run_code()